# Based on Age, Experience, predicting the salary

In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Practise').getOrCreate()

In [4]:
df_spark=spark.read.csv('test1.csv',header=True,inferSchema=True)

In [5]:
df_spark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
df_spark.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
# ['Age','Experience'] ---- grouped together ---- independent feature

In [11]:
from pyspark.ml.feature import VectorAssembler
asse=VectorAssembler(inputCols=['age','Experience'],outputCol='Independent features')

In [12]:
output=asse.transform(df_spark)

In [13]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [14]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent features']

In [15]:
final=output.select('Independent features','Salary')

In [16]:
final.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



# Train test split using RandomSplit

In [26]:
from pyspark.ml.regression import LinearRegression

In [28]:
train_data,test_data=final.randomSplit([0.75,0.25])
regg=LinearRegression(featuresCol='Independent features',labelCol='Salary')

In [30]:
regg=regg.fit(train_data)

In [31]:
regg.coefficients

DenseVector([28.4757, 1271.3568])

In [32]:
regg.intercept

14299.832495812996

In [34]:
pred_results=regg.evaluate(test_data)

In [36]:
pred_results.predictions.show()

C:\Users\kcjc608\Anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|         [31.0,10.0]| 30000|27896.147403685147|
+--------------------+------+------------------+



In [39]:
pred_results.meanAbsoluteError

2103.852596314853

In [40]:
pred_results.meanSquaredError

4426195.747020748